# CellRank Analysis of mouse intestinal crypt cells (TE001 and TE006)

### Import Libraries

In [1]:
import sys
import cellrank as cr
import scanpy as sc
import numpy as np
import pandas as pd
import os
from matplotlib import rc_context
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore", category=UserWarning)

### Parameters Setting

In [3]:
# a.1) setup path to data-containing folder and savings and parameters
base_path = "~/Clouds/Dropbox/isc-data-at-cell/"
input_dir = os.path.join(base_path,"input")
figures_dir = os.path.join(base_path,"figures")

sc.settings.set_figure_params(frameon=False, dpi=100)
cr.settings.verbosity = 2

n_macro_CytoTRACE = 8 # number of macrostates 

cytotrace_markers = ['Smarca5','Rbbp7','Tcerg1','Hnrnpd','Hmg20b','Nelfe','Ube2i','Etv5','Ubn1','Mbd3','Dek','Maz',
                     'Itgb3bp','Ilf2','Pa2g4'] # Id3','Hnf4g','Atoh1','Spdef','Neurod1' markers upregulated in cytotrace (Fig 1e) 

### Input Data Definition

In [ ]:
# Counts data (exported from Seurat@RNA assay)
counts_h5ad = input_dir + "TE001-counts.h5ad"
# Metadata for TE001 from VIPER analysis
metadata_csv = input_dir + "TE001-metadata-umap-and-clusters-for-paper.csv"
# Metadata for MRs lineage enrichment for TE001 (three lineages from step 1 of iterative clustering algorihtm)
mrs_enrichment_csv = h5ad_path + "TE001-mrs-enrichment-table-3D-plot.csv"

### Load Datasets and Processed Data

In [ ]:
adata = sc.read_h5ad(counts_h5ad) # load in object
metadata = pd.read_csv(metadata_csv)
mrs_enrichment = pd.read_csv(mrs_enrichment_csv)

# a.6) process metadata in adata
specified_columns = ["cell_id", "nCount_RNA", "nFeature_RNA", "mt_percent", "cytotrace_score.ges",
                     "cytotrace_gcs.ges", "S.Score", "G2M.Score", "Phase", "seurat_clusters", "singleR_labels", "stemness_index.ges"]

adata.obs = adata.obs[specified_columns]
cells_to_analyze = metadata['cell_id'] # cells to analyze
adata = adata[adata.obs_names.isin(cells_to_analyze)] # subset cells to analyze in adata

adata.obs = pd.merge(adata.obs, metadata, on='cell_id', how='left').set_index('cell_id') # merge metadata and include into counts object
adata.obs = pd.merge(adata.obs, mrs_enrichment, on='cell_id', how='left').set_index('cell_id') # merge metadata and include into counts object

adata.obs['seurat_clusters'] = adata.obs['seurat_clusters'].astype('category') # clusters as categorical variable

# a.7) set UMAP coordinates to those obtained at protein activty
umap_coordinates = np.array(adata.obs.loc[:, ['UMAP_1_scanpy','UMAP_2_scanpy']]) 
adata.obsm['X_umap'] = umap_coordinates

# a.8) set MR enrichment coordinates to those obtained at protein activty
enr1_coordinates = np.array(adata.obs.loc[:, ['absorptive_lineage','secretor_lineage']]) 
adata.obsm['X_enr1'] = enr1_coordinates

enr2_coordinates = np.array(adata.obs.loc[:, ['progenitors_lineage','secretor_lineage']]) 
adata.obsm['X_enr2'] = enr2_coordinates


# a.9) Include metadata of terminal states for CellRank analysis
adata.obs['terminal_states'] = adata.obs['iter_cluster_id_with_paneth']
adata.obs['terminal_states'].iloc[adata.obs['terminal_states'].isin(["stem-1","stem-2"])] = np.nan

print("adata contains the counts for the TE001 dataset")
counts = adata.raw.to_adata() # the adata already contains the counts matrix

## CellRank2 analysis with CytoTRACE Kernel
Import CellRank2 kernel, compute CytoTRACE score with CellRank (sparse), and compute transition matrix for CellRank2 analysis

### Processing counts for CellRank analysis

In [7]:
print("Preprocessing counts matrix for CellRank 2 analysis")
sc.tl.pca(adata, random_state=0)
sc.pp.neighbors(adata, random_state=0)

Preprocessing counts matrix for CellRank 2 analysis


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [8]:
### c) CytoTRACE kernel
# c.1) Setup kernel
print("Working with CytoTRACE kernel")
from cellrank.kernels import CytoTRACEKernel
import scvelo as scv
# CytoTRACE by default uses imputed data - a simple way to compute
# k-NN imputed data is to use scVelo's moments function.
# However, note that this function expects `spliced` counts because
# it's designed for RNA velocity, so we're using a simple hack here:
if 'spliced' not in adata.layers or 'unspliced' not in adata.layers:
    adata.layers['spliced'] = adata.X
    adata.layers['unspliced'] = adata.X
scv.pp.moments(adata) # hack for CytoTRACEkernel

ctk = CytoTRACEKernel(adata) # initialize the CellRank2 kernel

# c.2) compute transition matrix
ctk = ctk.compute_cytotrace().compute_transition_matrix(threshold_scheme="soft",nu=0.5) # compute transition matrix


Working with CytoTRACE kernel
Normalized count data: X.
computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:06) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
Computing CytoTRACE score with `15385` genes
Adding `adata.obs['ct_score']`
       `adata.obs['ct_pseudotime']`
       `adata.obs['ct_num_exp_genes']`
       `adata.var['ct_gene_corr']`
       `adata.var['ct_correlates']`
       `adata.uns['ct_params']`
    Finish (0:00:00)
Computing transition matrix based on pseudotime


/Users/lucazanella7/mambaforge/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:528: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


  0%|          | 0/3656 [00:00<?, ?cell/s]

    Finish (0:00:01)
